In [1]:
from bs4 import BeautifulSoup
import requests
import pickle
import pandas as pd
import csv  

In [2]:
df = pd.read_csv('../data/full_dataset.csv')

In [10]:
csvRatingsPath = '../data/web_scraped/food_ratings.csv'

In [4]:
rating_limit = 4.0

In [5]:
websites = set()
countDict = {}
for link in df["link"]:
  websites.add(link.split("/")[0])
  # if ("www.allrecipes.com" in link):
  #   counter += 1

for website in websites:
  countDict[website] = 0

for link in df["link"]:
  countDict[link.split("/")[0]] += 1

In [6]:
countDict

{'allrecipes.com': 3204,
 'online-cookbook.com': 5691,
 'www.chowhound.com': 5671,
 'www.lovefood.com': 1930,
 'www.foodgeeks.com': 8963,
 'cooking.nytimes.com': 16367,
 'www.cookstr.com': 8797,
 'cookpad.com': 61020,
 'www.yummly.com': 51963,
 'cookeatshare.com': 59307,
 'www.delish.com': 3880,
 'www.seriouseats.com': 12632,
 'www.epicurious.com': 129444,
 'www.foodandwine.com': 15436,
 'www.tasteofhome.com': 51594,
 'www.vegetariantimes.com': 4578,
 'www.foodrepublic.com': 2259,
 'recipeland.com': 24418,
 'tastykitchen.com': 78768,
 'www.foodnetwork.com': 49443,
 'www.myrecipes.com': 64895,
 'food52.com': 48501,
 'www.landolakes.com': 2492,
 'www.allrecipes.com': 61398,
 'www.food.com': 499616,
 'www.kraftrecipes.com': 42010,
 'www.cookbooks.com': 896341,
 'recipes-plus.com': 20524}

In [8]:
foodLinks = []
for link in df["link"]:
  if ("www.food.com" in link):
    foodLinks.append(link)

In [9]:
foodLinks

['www.food.com/recipe/kale-carrot-and-apple-calcium-booster-juice-for-juicer-459269',
 'www.food.com/recipe/chocolade-pudding-chocolate-pudding-233737',
 'www.food.com/recipe/haluski-407129',
 'www.food.com/recipe/cajun-style-white-bean-soup-503131',
 'www.food.com/recipe/toddler-tamer-tea-188781',
 'www.food.com/recipe/hummus-no-fat-450362',
 'www.food.com/recipe/mount-olympus-cocktail-305014',
 'www.food.com/recipe/risotto-coi-frutti-di-mare-risotto-with-seafood-284538',
 'www.food.com/recipe/spinach-with-sesame-and-miso-209003',
 'www.food.com/recipe/easy-raw-chocolate-bark-497280',
 'www.food.com/recipe/indian-ice-cream-cake-468760',
 'www.food.com/recipe/green-tea-ice-pops-44157',
 'www.food.com/recipe/chicken-in-phyllo-47950',
 'www.food.com/recipe/snowman-sugar-cookies-with-frosting-109544',
 'www.food.com/recipe/college-sandwich-110016',
 'www.food.com/recipe/savory-roast-leg-of-lamb-147163',
 'www.food.com/recipe/cheesy-garlic-bread-295491',
 'www.food.com/recipe/award-winning

In [28]:
try:
  df = pd.read_csv(csvRatingsPath)
  foodLinksWithRatings = []
  foodLinksWithRatings.extend(df.values.tolist())
except:
  foodLinksWithRatings = []

In [29]:
header = ['link', 'rating']
def getFoodRatings(originalLink, idx):
  link = f'https://{originalLink}'

  page = requests.get(link)
  soup = BeautifulSoup(page.content, 'lxml')
  div = soup.find("div", {"class": "five-star-rating"})
  rating = 0
  if div:
    svg = div.findAll("svg", {"width": "18px"})
    for star in svg:
      linearGradient = star.find("lineargradient")
      stop = linearGradient.find("stop", {"stop-color": "#9C1F1E"})
      rating += round(float(stop["offset"][:-1])/100, 2)
    foodLinksWithRatings.append([originalLink, round(float(rating), 2)])
  else:
    foodLinksWithRatings.append([originalLink, 0.0])

  if (idx % 100 == 0):
    print(idx)
    with open(csvRatingsPath, 'w') as f:
      write = csv.writer(f)
      write.writerow(header)
      write.writerows(foodLinksWithRatings)

for idx in range(len(foodLinksWithRatings), len(foodLinks)):
  getFoodRatings(foodLinks[idx], idx)

46400
46500
46600
46700
46800
46900
47000
47100
47200
47300
47400
47500
47600
47700
47800
